In [17]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\90536\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [18]:
#SQLite için gerekli paketi çağıralım
import sqlite3

In [19]:
#veri tabanına bağlanalım ve bağlantıyı bir değişkene yükleyelim
conn = sqlite3.connect("db")

In [20]:
# Veri tabanı ile iletişim sağlayacak bir Cursor elemanı oluşturalım
cursor = conn.cursor()

In [21]:
#SQL komutumuzu cursor kullanarak çalıştıralım
cursor.execute("CREATE TABLE Pozisyon(id INTEGER PRIMARY KEY, calisan_id INTEGER NOT NULL, unvan VARCHAR(255), baslangicTarihi INTEGER, bitisTarihi INTEGER)")

OperationalError: table Pozisyon already exists

In [22]:
# Veri tabanında yer alan tabloları listeleyen SQL kodunu çalıştıralım
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")

#SQL sonucunda oluşan yanıtı alarak tables değişkenine yükleyelim
tables = cursor.fetchall()

# tables değişkeni içerisinde neler olduğuna bakalım
print(tables)

[('Calisan',), ('Bolum',), ('Pozisyon',)]


In [23]:
# Yapılan değişiklikleri veri tabanı üzerine kaydedelim
conn.commit()

#Veri tabanı bağlantısını kapatalım
conn.close()

In [49]:
#SQLAlchemy’i, ilgili alt paketleriyle birlikte çağıralım
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

In [50]:
# Veri tabanı motoru oluştur, veri tabanı bağlantısını sağladık
engine = create_engine("sqlite:///db")
Base = declarative_base()

C:\Users\90536\AppData\Local\Temp\ipykernel_11432\3891686630.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [48]:
class Yoneticilik(Base):
    #__tablename__ değişkeni, bu sınıfın hangi tablo ile birlikte çalıştığını tanımlar
    __tablename__ = "Yoneticilik"
    id = Column(Integer, primary_key=True)
    calisan_id = Column(Integer)
    bolum_id = Column(Integer)
    baslangicTarihi = Column(Date)


In [29]:
#Bu kod bloğu çalıştırıldığında tanımlanan sınıflar veri tabanı üzerinde de oluşturulur ve eşleştirmeler tamamlanır
Base.metadata.create_all(engine)

In [30]:
class BolumCalisan(Base):
    #__tablename__ değişkeni, bu sınıfın hangi tablo ile birlikte çalıştığını tanımlar
    __tablename__ = "BolumCalisan"
    id = Column(Integer, primary_key=True)
    calisan_id = Column(Integer)
    bolum_id = Column(Integer)
    baslangicTarihi = Column(Date)
    bitisTarihi = Column(Date)

In [31]:
Base.metadata.create_all(engine)

In [51]:
class Calisan(Base):
    __tablename__ = "Calisan"
    id = Column(Integer, primary_key=True)
    adi = Column(String)
    soyadi = Column(String)
    cinsiyet = Column(String)
    iseGirisTarihi = Column(Date)

    #ilişki oluşturuyoruz
    pozisyon = relationship("Pozisyon")
    maas = relationship("Maas")
    yoneticilik = relationship("Pozisyon")
    bolumCalisan = relationship("BolumCalisan")

In [52]:
class Bolum(Base):
    __tablename__ = "Bolum"
    id = Column(Integer, primary_key=True)
    adi = Column(String)

In [53]:
class Pozisyon(Base):
    __tablename__ = "Pozisyon"
    id = Column(Integer, primary_key=True)
    calisan_id = Column(Integer, ForeignKey("Calisan.id"))
    unvan = Column(String)
    baslangicTarihi = Column(Date)
    bitisTarihi = Column(Date)

    calisan = relationship("Calisan", back_populates="pozisyon")

In [54]:
class Maas(Base):
    __tablename__ = "Maas"
    id = Column(Integer, primary_key=True)
    calisan_id = Column(Integer, ForeignKey("Calisan.id"))
    unvan = Column(String)
    baslangicTarihi = Column(Date)
    bitisTarihi = Column(Date)

    calisan = relationship("Calisan", back_populates="maas")

In [55]:
class Yoneticilik(Base):
    #__tablename__ değişkeni, bu sınıfın hangi tablo ile birlikte çalıştığını tanımlar
    __table_args__ = {'extend_existing': True} #Redefine
    __tablename__ = "Yoneticilik"
    id = Column(Integer, primary_key=True)
    calisan_id = Column(Integer, ForeignKey("Calisan.id"))
    bolum_id = Column(Integer, ForeignKey("Bolum.id"))
    baslangicTarihi = Column(Date)
    bitisTarihi = Column(Date)

    calisan = relationship("Calisan", back_populates="yoneticilik")
    bolum = relationship("Bolum", back_populates="yoneticilik")

In [56]:
class BolumCalısan(Base):
    #__tablename__ değişkeni, bu sınıfın hangi tablo ile birlikte çalıştığını tanımlar
    __table_args__ = {'extend_existing': True} #Redefine
    __tablename__ = "BolumCalısan"
    id = Column(Integer, primary_key=True)
    calisan_id = Column(Integer, ForeignKey("Calisan.id"))
    bolum_id = Column(Integer, ForeignKey("Bolum.id"))
    baslangicTarihi = Column(Date)
    bitisTarihi = Column(Date)

    calisan = relationship("Calisan", back_populates="bolumcalisan")
    bolum = relationship("Bolum", back_populates="bolumcalisan")

In [57]:
Base.metadata.create_all(engine)